#### <b>Managing the conversation History </b>

one important concept to understand when building chatbots is how to manage conversation history.if left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the llm. therefore, it is important to add a step that limits the size of the messages you are passing in.

- <b>trim_messages</b> : helper to reduce how many messages we're sending to the model. the trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [1]:
import os
from dotenv import load_dotenv 
from rich import print 
load_dotenv(dotenv_path="../.env")

from langchain_groq import ChatGroq 


os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=ChatGroq(model="gemma2-9b-it")

In [19]:
from langchain_core.messages import SystemMessage,trim_messages,HumanMessage,AIMessage
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [20]:
messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! i am abhijit"),
    AIMessage(content="hi"),
    HumanMessage(content="i like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
     HumanMessage(content=" having fun ?"),
    AIMessage(content="yes!"),
]

In [ ]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content=' having fun ?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [21]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",("You are helpful assistant.Answer all the questions to the best of your ability in {language}")),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [22]:
from operator import itemgetter 

from langchain_core.runnables import RunnablePassthrough 

chain=(
        RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
        |prompt
        |llm
)

In [23]:
print(chain.invoke(
    {
    "messages":messages+[HumanMessage(content="What ice cream do i like")],
    "language":"english"
    }
))

AIMessage(
    content="As an AI, I don't have access to your personal information, including your ice cream 
preferences!\n\nWhat's your favorite flavor? 😊  \n",
    additional_kwargs={},
    response_metadata={
        'token_usage': {
            'completion_tokens': 34,
            'prompt_tokens': 85,
            'total_tokens': 119,
            'completion_time': 0.061818182,
            'prompt_time': 0.002573999,
            'queue_time': 0.246618011,
            'total_time': 0.064392181
        },
        'model_name': 'gemma2-9b-it',
        'system_fingerprint': 'fp_10c08bf97d',
        'service_tier': 'on_demand',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--cb013430-7fc1-4fa2-82a4-d849b8d0da4d-0',
    usage_metadata={'input_tokens': 85, 'output_tokens': 34, 'total_tokens': 119}
)